In [104]:
import requests
from geopy.geocoders import Nominatim
import folium
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
from sklearn.cluster import KMeans

In [51]:
# find all the cities in argentina with more than 50K people
df = pd.read_csv('argentina.csv')
df = df[df['population_proper'] > 50000].loc[:,['city', 'lat', 'lng', 'capital', 'population_proper']]

In [57]:
df[df['capital'] == 'admin']

,city,lat,lng,capital,population_proper
1,Córdoba,-31.4167,-64.1833,admin,1329604.0
3,San Miguel de Tucumán,-26.8167,-65.2167,admin,605767.0
5,Salta,-24.7883,-65.4106,admin,535303.0
6,San Juan,-31.5375,-68.5364,admin,471389.0
8,Santa Fe,-31.6333,-60.7000,admin,405683.0
9,Corrientes,-27.4833,-58.8167,admin,352646.0
10,San Salvador de Jujuy,-24.1856,-65.2994,admin,321789.0
12,Resistencia,-27.4514,-58.9867,admin,291720.0
13,Posadas,-27.3667,-55.8969,admin,275988.0
14,Santiago del Estero,-27.7844,-64.2669,admin,252192.0


In [73]:
address = 'puerto madrin, argentina'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Argentina are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Argentina are -42.7729674, -65.0366104.


In [77]:
list_size = df['population_proper'] / df['population_proper'].max()
df['pop_norm'] = list_size
df

,city,lat,lng,capital,population_proper,pop_norm
0,Buenos Aires,-34.5997,-58.3819,primary,3054300.0,1.000000
1,Córdoba,-31.4167,-64.1833,admin,1329604.0,0.435322
2,Rosario,-32.9575,-60.6394,minor,1193605.0,0.390795
3,San Miguel de Tucumán,-26.8167,-65.2167,admin,605767.0,0.198333
4,Mar del Plata,-38.0000,-57.5500,minor,593337.0,0.194263
...,...,...,...,...,...,...
68,San Pedro,-24.2196,-64.8700,minor,52068.0,0.017047
69,Punta Alta,-38.8800,-62.0800,minor,54730.0,0.017919
71,Ushuaia,-54.8072,-68.3044,admin,56956.0,0.018648
72,Azul,-36.7833,-59.8500,minor,55728.0,0.018246


In [81]:
map_argentina = folium.Map(location=[latitude, longitude], zoom_start=3.5)

# add markers to df
for lat, lng, city, pop, size in zip(df['lat'], df['lng'], df['city'], df['population_proper'], df['pop_norm']):
    label = '{},\n {}'.format(city, pop)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5 * size + 3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_argentina)  
    
map_argentina

In [7]:
CLIENT_ID = 'N0YWYR2EWEA2GRUSN24MBCLHF3F4P3L2KNBMUHGVEVSB1FRE' # your Foursquare ID
CLIENT_SECRET = 'KX25L0IQX0OVH3X1BYFA0YQF23V4BKFXX5WOFNPQ5GKNJNFX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N0YWYR2EWEA2GRUSN24MBCLHF3F4P3L2KNBMUHGVEVSB1FRE
CLIENT_SECRET:KX25L0IQX0OVH3X1BYFA0YQF23V4BKFXX5WOFNPQ5GKNJNFX


In [83]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
argentina_venues = getNearbyVenues(df['city'],  df['lat'], df['lng'], radius=1000)

Buenos Aires
Córdoba
Rosario
San Miguel de Tucumán
Mar del Plata
Salta
San Juan
Lanús
Santa Fe
Corrientes
San Salvador de Jujuy
Bahía Blanca
Resistencia
Posadas
Santiago del Estero
Paraná
Merlo
Neuquén
Quilmes
Formosa
José C. Paz
Godoy Cruz
La Plata
La Rioja
Berazategui
Comodoro Rivadavia
San Luis
José María Ezeiza
San Nicolás de los Arroyos
Catamarca
Concordia
Florencio Varela
San Justo
San Martín
Tandil
Puerto Madryn
Mendoza
San Carlos de Bariloche
Villa Mercedes
Gualeguaychú
San Rafael
La Banda
Santa Rosa
Berisso
Morón
Zárate
Río Gallegos
Caseros
Rafaela
Pergamino
Campana
Presidencia Roque Sáenz Peña
Luján
Pilar
Necochea
Villa María
San Ramón de la Nueva Orán
Concepción del Uruguay
Goya
Reconquista
Las Heras
Oberá
San Francisco
Los Polvorines
Tartagal
San Pedro
Punta Alta
Ushuaia
Azul
Cruz del Eje


In [94]:
argentina_venues.rename(columns={'Neighborhood Latitude':'Latitude', 'Neighborhood Longitude': 'Longitude'}, inplace=True)

In [95]:
argentina_venues

,city,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Buenos Aires,-34.5997,-58.3819,Teatro Colón,-34.601122,-58.383139,Opera House
1,Buenos Aires,-34.5997,-58.3819,Teatro Nacional Cervantes,-34.599408,-58.383880,Theater
2,Buenos Aires,-34.5997,-58.3819,Santos Manjares,-34.597867,-58.380387,BBQ Joint
3,Buenos Aires,-34.5997,-58.3819,NEGRO. Cueva de café,-34.600848,-58.379612,Coffee Shop
4,Buenos Aires,-34.5997,-58.3819,Delhi Mahal,-34.599196,-58.383107,Indian Restaurant
...,...,...,...,...,...,...,...
1424,Azul,-36.7833,-59.8500,Cooperativa Obrera,-36.788078,-59.845718,Department Store
1425,Cruz del Eje,-30.7333,-64.8000,Petrobras Cruz Del Eje,-30.726202,-64.802808,Café
1426,Cruz del Eje,-30.7333,-64.8000,Viejo Munich,-30.736909,-64.791571,German Restaurant
1427,Cruz del Eje,-30.7333,-64.8000,El Cardon,-30.737260,-64.791606,BBQ Joint


In [89]:
argentina_venues.groupby('city').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
city,,,,,,
Azul,4,4,4,4,4,4
Bahía Blanca,34,34,34,34,34,34
Berazategui,12,12,12,12,12,12
Berisso,6,6,6,6,6,6
Buenos Aires,100,100,100,100,100,100
...,...,...,...,...,...,...
Tandil,18,18,18,18,18,18
Ushuaia,99,99,99,99,99,99
Villa María,10,10,10,10,10,10


In [91]:
print('There are {} uniques categories.'.format(len(argentina_venues['Venue Category'].unique())))

There are 188 uniques categories.


In [97]:
# one hot encoding
argentina_onehot = pd.get_dummies(argentina_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
argentina_onehot['city'] = argentina_venues['city'] 

# move neighborhood column to the first column
fixed_columns = [argentina_onehot.columns[-1]] + list(argentina_onehot.columns[:-1])
argentina_onehot = argentina_onehot[fixed_columns]

argentina_onehot.shape

(1429, 189)

In [98]:
argentina_grouped = argentina_onehot.groupby('city').mean().reset_index()
argentina_grouped

,city,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Waterfront,Wine Bar,Wine Shop,Women's Store
0,Azul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
1,Bahía Blanca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.088235,0.000000,...,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
2,Berazategui,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.000000,...,0.083333,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
3,Berisso,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
4,Buenos Aires,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040000,0.010000,...,0.000000,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.02,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Tandil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
65,Ushuaia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.121212,0.010101,...,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
66,Villa María,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
67,Villa Mercedes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.166667,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0


In [127]:
num_top_venues = 10

for city in argentina_grouped['city']:
    print("----"+city+"----")
    temp = argentina_grouped[argentina_grouped['city'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Azul----
                           venue  freq
0                    Bus Station  0.50
1                            Pub  0.25
2               Department Store  0.25
3             African Restaurant  0.00
4            Peruvian Restaurant  0.00
5                    Opera House  0.00
6              Other Repair Shop  0.00
7          Outdoors & Recreation  0.00
8  Paper / Office Supplies Store  0.00
9                           Park  0.00


----Bahía Blanca----
                    venue  freq
0                    Café  0.12
1                   Hotel  0.09
2  Argentinian Restaurant  0.09
3                 Brewery  0.06
4      Italian Restaurant  0.06
5               Gastropub  0.06
6                 Theater  0.06
7             Gas Station  0.03
8        Tapas Restaurant  0.03
9             Coffee Shop  0.03


----Berazategui----
                    venue  freq
0    Fast Food Restaurant  0.17
1          Ice Cream Shop  0.17
2                Pharmacy  0.08
3                    Café  0.08
4

                    venue  freq
0             Pizza Place  0.17
1               Nightclub  0.17
2      Italian Restaurant  0.08
3            Burger Joint  0.08
4           Grocery Store  0.08
5      Athletics & Sports  0.08
6          Sandwich Place  0.08
7           Shopping Mall  0.08
8  Argentinian Restaurant  0.08
9                   Plaza  0.08


----Posadas----
                    venue  freq
0                   Hotel  0.13
1                    Café  0.08
2           Shopping Mall  0.05
3                Pharmacy  0.05
4              Restaurant  0.05
5  Argentinian Restaurant  0.05
6           Grocery Store  0.05
7                   Plaza  0.05
8               BBQ Joint  0.05
9           Bowling Alley  0.03


----Presidencia Roque Sáenz Peña----
                   venue  freq
0     Italian Restaurant  0.17
1         Ice Cream Shop  0.17
2            Supermarket  0.17
3                  Plaza  0.17
4               Pharmacy  0.17
5                    Gym  0.17
6     African Restaura

In [134]:
list_of_venues = argentina_venues['Venue Category'].unique()
list_of_venues

array(['Opera House', 'Theater', 'BBQ Joint', 'Coffee Shop',
       'Indian Restaurant', 'Gym / Fitness Center', 'Peruvian Restaurant',
       'Pizza Place', 'Wine Shop', 'Hotel', 'Italian Restaurant',
       'Bakery', 'Salad Place', 'Monument / Landmark', 'Plaza', 'Brewery',
       'Argentinian Restaurant', 'Hotel Bar', 'Asian Restaurant', 'Café',
       'Korean Restaurant', 'Spanish Restaurant', 'Ice Cream Shop',
       'Creperie', 'Lounge', 'Cocktail Bar', 'Sandwich Place', 'Pub',
       'Video Game Store', 'Venezuelan Restaurant',
       'Comfort Food Restaurant', 'Nail Salon', 'Shopping Mall',
       'Restaurant', 'Mediterranean Restaurant', 'Smoke Shop',
       'Pet Store', 'German Restaurant', 'Bar', 'Art Gallery', 'Beer Bar',
       'Diner', 'Chinese Restaurant', 'Deli / Bodega', 'Burger Joint',
       'Gay Bar', 'French Restaurant', 'History Museum', 'Gym',
       'Food & Drink Shop', 'Movie Theater', 'Bookstore',
       'Seafood Restaurant', 'Turkish Restaurant', 'Fast Food R

In [145]:
matchers = ['argentinian','bbq', 'restaurant']
matching = [s.lower() for s in list_of_venues if any(xs in s.lower() for xs in matchers)]
matching

['bbq joint',
 'indian restaurant',
 'peruvian restaurant',
 'italian restaurant',
 'argentinian restaurant',
 'asian restaurant',
 'korean restaurant',
 'spanish restaurant',
 'venezuelan restaurant',
 'comfort food restaurant',
 'restaurant',
 'mediterranean restaurant',
 'german restaurant',
 'chinese restaurant',
 'french restaurant',
 'seafood restaurant',
 'turkish restaurant',
 'fast food restaurant',
 'middle eastern restaurant',
 'sushi restaurant',
 'american restaurant',
 'mexican restaurant',
 'african restaurant',
 'latin american restaurant',
 'empanada restaurant',
 'vegetarian / vegan restaurant',
 'tapas restaurant',
 'theme restaurant']

In [139]:
for venue in list_of_venues:
    if 'argentinian' or 'bbq' in venue.lower():
        print(venue.lower())
#     if 'bbq' in venue.lower():
#         print(venue.lower())

opera house
theater
bbq joint
coffee shop
indian restaurant
gym / fitness center
peruvian restaurant
pizza place
wine shop
hotel
italian restaurant
bakery
salad place
monument / landmark
plaza
brewery
argentinian restaurant
hotel bar
asian restaurant
café
korean restaurant
spanish restaurant
ice cream shop
creperie
lounge
cocktail bar
sandwich place
pub
video game store
venezuelan restaurant
comfort food restaurant
nail salon
shopping mall
restaurant
mediterranean restaurant
smoke shop
pet store
german restaurant
bar
art gallery
beer bar
diner
chinese restaurant
deli / bodega
burger joint
gay bar
french restaurant
history museum
gym
food & drink shop
movie theater
bookstore
seafood restaurant
turkish restaurant
fast food restaurant
convenience store
nightclub
market
river
historic site
middle eastern restaurant
gym pool
cheese shop
sushi restaurant
tattoo parlor
american restaurant
mexican restaurant
hostel
pharmacy
candy store
african restaurant
shopping plaza
breakfast spot
gastropub

In [101]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [103]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['city'] = argentina_grouped['city']

for ind in np.arange(argentina_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(argentina_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Azul,Bus Station,Pub,Department Store,Electronics Store,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Event Service
1,Bahía Blanca,Café,Hotel,Argentinian Restaurant,Brewery,Italian Restaurant,Gastropub,Theater,Grocery Store,Sandwich Place,Burger Joint
2,Berazategui,Ice Cream Shop,Fast Food Restaurant,Argentinian Restaurant,Bakery,Café,Chinese Restaurant,Train Station,Pedestrian Plaza,Pharmacy,Pizza Place
3,Berisso,Movie Theater,Harbor / Marina,River,Pet Store,Basketball Stadium,Convenience Store,Health Food Store,Diner,Flea Market,Fish Market
4,Buenos Aires,Coffee Shop,Theater,Café,Hotel,Restaurant,Pizza Place,Argentinian Restaurant,BBQ Joint,Ice Cream Shop,Cocktail Bar


In [105]:
# set number of clusters
kclusters = 5

argentina_grouped_clustering = argentina_grouped.drop('city', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(argentina_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([1, 4, 4, 4, 4, 3, 4, 2, 4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 4, 4])

In [107]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [111]:
argentina_merged = df

# merge argentina_grouped with argentina_data to add latitude/longitude for each neighborhood
argentina_merged = argentina_merged.join(city_venues_sorted.set_index('city'), on='city')

argentina_merged.head() # check the last columns!

,city,lat,lng,capital,population_proper,pop_norm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Buenos Aires,-34.5997,-58.3819,primary,3054300.0,1.000000,4.0,Coffee Shop,Theater,Café,Hotel,Restaurant,Pizza Place,Argentinian Restaurant,BBQ Joint,Ice Cream Shop,Cocktail Bar
1,Córdoba,-31.4167,-64.1833,admin,1329604.0,0.435322,2.0,Hotel,Coffee Shop,Sandwich Place,Café,Restaurant,Plaza,Argentinian Restaurant,Gym,Theater,Nightclub
2,Rosario,-32.9575,-60.6394,minor,1193605.0,0.390795,4.0,Ice Cream Shop,Argentinian Restaurant,Pizza Place,Café,Bar,Bakery,Brewery,Gym,Hotel,Bed & Breakfast
3,San Miguel de Tucumán,-26.8167,-65.2167,admin,605767.0,0.198333,4.0,Nightclub,Plaza,Soccer Field,Bar,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Breakfast Spot,Beer Garden
4,Mar del Plata,-38.0000,-57.5500,minor,593337.0,0.194263,4.0,Café,Restaurant,Pizza Place,Theater,Ice Cream Shop,Burger Joint,Dessert Shop,Plaza,Hotel,Coffee Shop


In [113]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(argentina_merged['lat'], argentina_merged['lng'], argentina_merged['city'], argentina_merged['Cluster Labels']):
    try:
        label = folium.Popup(str(poi) + ' Cluster ' + str(int(cluster)), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
    except:
        print(lat, lon, poi, "------>   Has no near venues")
map_clusters

-22.5 -63.8333 Tartagal ------>   Has no near venues


In [119]:
argentina_merged.loc[argentina_merged['Cluster Labels'] == 0, argentina_merged.columns[[0] + list(range(5, argentina_merged.shape[1]))]]

,city,pop_norm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Rafaela,0.030431,0.0,Furniture / Home Store,Market,French Restaurant,Food Court,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Event Service
63,Oberá,0.021646,0.0,Furniture / Home Store,Bookstore,Coffee Shop,Event Service,Empanada Restaurant,Food Court,Food & Drink Shop,Food,Flea Market,Fish Market
64,San Francisco,0.020299,0.0,Furniture / Home Store,Clothing Store,Electronics Store,Construction & Landscaping,Entertainment Service,Food Service,Food Court,Food & Drink Shop,Food,Flea Market


In [120]:
argentina_merged.loc[argentina_merged['Cluster Labels'] == 1, argentina_merged.columns[[0] + list(range(5, argentina_merged.shape[1]))]]

,city,pop_norm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Reconquista,0.023098,1.0,Ice Cream Shop,Bowling Alley,Breakfast Spot,Nightclub,Bus Station,Women's Store,Entertainment Service,Food,Flea Market,Fish Market
68,San Pedro,0.017047,1.0,Bus Station,Ice Cream Shop,Bank,French Restaurant,Food Court,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant
72,Azul,0.018246,1.0,Bus Station,Pub,Department Store,Electronics Store,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Event Service


In [121]:
argentina_merged.loc[argentina_merged['Cluster Labels'] == 2, argentina_merged.columns[[0] + list(range(5, argentina_merged.shape[1]))]]

,city,pop_norm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Córdoba,0.435322,2.0,Hotel,Coffee Shop,Sandwich Place,Café,Restaurant,Plaza,Argentinian Restaurant,Gym,Theater,Nightclub
5,Salta,0.175262,2.0,Hotel,Argentinian Restaurant,Restaurant,Café,History Museum,Plaza,Hostel,Bar,Dessert Shop,Comfort Food Restaurant
13,Posadas,0.090360,2.0,Hotel,Café,Shopping Mall,Pharmacy,Argentinian Restaurant,Restaurant,Plaza,BBQ Joint,Grocery Store,Breakfast Spot
14,Santiago del Estero,0.082569,2.0,Hotel,Coffee Shop,Pub,Hotel Bar,Ice Cream Shop,Plaza,Bus Station,Italian Restaurant,Men's Store,Cocktail Bar
17,Neuquén,0.075696,2.0,Hotel,Fast Food Restaurant,Electronics Store,Argentinian Restaurant,Art Museum,Women's Store,Food Court,Food & Drink Shop,Food,Flea Market
26,San Luis,0.055642,2.0,Hotel,Café,Argentinian Restaurant,Pharmacy,Athletics & Sports,Coffee Shop,Restaurant,Bus Station,Plaza,Mexican Restaurant
28,San Nicolás de los Arroyos,0.052385,2.0,Hotel,Park,Restaurant,Café,Supermarket,Ice Cream Shop,Outdoors & Recreation,Plaza,Coffee Shop,Sports Club
29,Catamarca,0.052103,2.0,Restaurant,Argentinian Restaurant,Plaza,Hotel,BBQ Joint,Department Store,Coffee Shop,Diner,Pedestrian Plaza,Café
30,Concordia,0.048931,2.0,Hotel,Café,Plaza,Jewelry Store,Karaoke Bar,Bus Line,Grocery Store,Food,Flea Market,Fish Market
34,Tandil,0.038279,2.0,Hotel,Restaurant,Supermarket,Tea Room,Gym / Fitness Center,Fast Food Restaurant,Nightclub,Pizza Place,Plaza,Cheese Shop


In [122]:
argentina_merged.loc[argentina_merged['Cluster Labels'] == 3, argentina_merged.columns[[0] + list(range(5, argentina_merged.shape[1]))]]

,city,pop_norm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Campana,0.028439,3.0,Port,Women's Store,Duty-free Shop,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Event Service,Entertainment Service


In [123]:
argentina_merged.loc[argentina_merged['Cluster Labels'] == 4, argentina_merged.columns[[0] + list(range(5, argentina_merged.shape[1]))]]

,city,pop_norm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Buenos Aires,1.000000,4.0,Coffee Shop,Theater,Café,Hotel,Restaurant,Pizza Place,Argentinian Restaurant,BBQ Joint,Ice Cream Shop,Cocktail Bar
2,Rosario,0.390795,4.0,Ice Cream Shop,Argentinian Restaurant,Pizza Place,Café,Bar,Bakery,Brewery,Gym,Hotel,Bed & Breakfast
3,San Miguel de Tucumán,0.198333,4.0,Nightclub,Plaza,Soccer Field,Bar,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Breakfast Spot,Beer Garden
4,Mar del Plata,0.194263,4.0,Café,Restaurant,Pizza Place,Theater,Ice Cream Shop,Burger Joint,Dessert Shop,Plaza,Hotel,Coffee Shop
6,San Juan,0.154336,4.0,Restaurant,Diner,Hotel,Bakery,Pizza Place,Concert Hall,Coffee Shop,Park,Museum,Food
7,Lanús,0.150366,4.0,Ice Cream Shop,Argentinian Restaurant,Pizza Place,Bar,Soccer Field,Middle Eastern Restaurant,Memorial Site,Rock Club,Sushi Restaurant,Nightclub
8,Santa Fe,0.132824,4.0,Café,Ice Cream Shop,Bar,Argentinian Restaurant,Breakfast Spot,Plaza,Deli / Bodega,Coffee Shop,Sandwich Place,Burger Joint
9,Corrientes,0.115459,4.0,BBQ Joint,Ice Cream Shop,Gas Station,Supermarket,Multiplex,Women's Store,Food Court,Food & Drink Shop,Food,Flea Market
10,San Salvador de Jujuy,0.105356,4.0,Pizza Place,Argentinian Restaurant,Breakfast Spot,Coffee Shop,Restaurant,Grocery Store,Plaza,Pharmacy,Hostel,Café
11,Bahía Blanca,0.097928,4.0,Café,Hotel,Argentinian Restaurant,Brewery,Italian Restaurant,Gastropub,Theater,Grocery Store,Sandwich Place,Burger Joint
